In [1]:
import anvil.server
import anvil.media
from anvil.tables import app_tables
from anvil import BlobMedia
anvil.server.connect("4LWTNNDYE2AJJJCWPGPFAFHX-CLFFCZ4AB5DTBE3B")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [7]:
import pipreqs
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

In [3]:
import re
import cv2 
from pathlib import Path
import pytesseract as pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\\Users\\JCheng10\\AppData\\Local\\Programs\\Tesseract-OCR\\tesseract.exe'
import tensorflow as tf
from PIL import Image
import sys
from pdf2image import convert_from_path
import os
from io import StringIO
import imutils

In [26]:
def pdf_img(file, name):
    pages = convert_from_path(file, 600)
    image_counter=0
    for page in pages:
        image_counter += 1
        filename = name[:-4]+"_"+str(image_counter)+".jpg"

        # Save the image of the page in system
        page.save(filename, 'JPEG')

        # Increment the counter to update filename
        
        
    return image_counter

def crop(img, pge):
    
    kernel1 = np.ones((3,5),np.uint8)
    kernel2 = np.ones((9,9),np.uint8)

    imgGray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBW=cv2.threshold(imgGray, 230, 255, cv2.THRESH_BINARY_INV)[1]

    img1=cv2.erode(imgBW, kernel1, iterations=1)
    img2=cv2.dilate(img1, kernel2, iterations=3)

    #plt.imshow(img2)
    #plt.show()

    imgLines= cv2.HoughLinesP(img2,15,np.pi/180,10, minLineLength = 3000, maxLineGap = 100)

    print(imgLines)
    for i in range(len(imgLines)):
        for x1,y1,x2,y2 in imgLines[i]:
            cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
        
    #plt.imshow(img)
    #plt.show()
    h, w, c = img.shape
    roi = img[imgLines[0][0][1]:, :-600]
    cv2.imwrite('roi'+ str(pge)+'.jpg',roi)
    
    return

# noise removal
def preprocess(image):
    image = cv2.medianBlur(image,5)

    kernel = np.ones((5,5),np.uint8)
    image = cv2.dilate(image, kernel, iterations = 1)
    
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)


def output_file(image_lst, name):
    # Creating a text file to write the output
    outfile = name[:-4] + ".txt"
    
    # Open the file in append mode so that 
    # All contents of all images are added to the same file
    f = open(outfile, "w")
    custom_config = r'--oem 1 --psm 6' #can play around with this
    text = ''
    for i in image_lst:
        text = text + str(((pytesseract.image_to_string(i, config=custom_config))))

    f.write(text)

    # Close the file after writing all the text.
    f.close()
    
    return text

def get_df(outfile):
    pn = []
    hn = []
    name = []
    qty = []
    count_pn = 0
    count_hn = 0
    with open (outfile, 'rt') as myfile:  # Open lorem.txt for reading text
        for myline in myfile:              # For each line, read to a string,
            myline = myline.strip('\n')
            lst = myline.split()
            
            if count_hn > count_pn: #check for subcomponent with multiple heat numbers
                last = pn[-1]
                pn.append(last)
                last = name[-1]
                name.append(last)
                count_pn += 1
                
            if len(lst) == 0:
                continue
            elif re.search("W-G", myline):
            #elif myline[0] == 'W' and myline[1] == '-':
                hn.append(lst[1])
                qty.append(lst[2])
                count_hn += 1
                continue
                
            a = re.search(".{4,5}\-([0-9]){3,4}\-([0-9]){4,5}", myline)
            b = re.search("[0-9]{9}", myline)
            c = re.search("[A-Z][0-9]{5,9}", myline)
            if bool(a) or bool(b) or bool(c):
                full = ' '.join(lst[1:-2])
                name.append(full)
                if bool(a):
                    pn.append(a.group(0))
                elif bool(b):
                    pn.append(b.group(0))
                else:
                    pn.append(c.group(0))
                count_pn += 1
            
    df = pd.DataFrame(list(zip(name, pn, hn, qty)),
               columns =['Name','Part Number', 'Heat Number', 'Quantity'])
    return df

In [ ]:
@anvil.server.callable
def extract(file, name):
    #anvil.media.download(wo)
    with anvil.media.TempFile(file) as filepath:
        pages = pdf_img(filepath, name)
    print("Gotten pages!")
    img = []
    for i in range(pages): 
        img.append(cv2.imread(name[:-4]+'_' + str(i+1) +'.jpg'))
    print('Converted and saved WO as image!')
    
    for i in range(len(img)):
        crop(img[i], i+1)
    img = []
    print('Cropped images!')
    
    for i in range(pages):
        pic = cv2.imread('roi' + str(i+1) +'.jpg')
        img.append(preprocess(pic))
    print('Extracted text! Putting in dataframe!')
    
    out = output_file(img, name)
    df = get_df(name[:-4]+".txt")
    print("Dataframe created!")
    
    for i in range(len(df)):
        app_tables.wo_components.add_row(Name = df['Name'][i], PN = df['Part Number'][i], HN = df['Heat Number'][i],
                          Qty = df['Quantity'][i])
    
    return True
    

@anvil.server.callable
def output_excel(name):
    datafile = app_tables.wo_components.search().to_csv()
    
    with anvil.media.TempFile(datafile) as filename:
        df = pd.read_csv(filename)
        df.drop('ID', axis = 1, inplace = True)
        print(df)
    print("Read in datatable!")
    
    path_to_desktop = str(os.path.join(Path.home(), "Desktop"))
    df.to_excel(path_to_desktop + '\\' + name[:-4] + '.xlsx', sheet_name='Excel Report', index = False)
    print('Excel created. Success!')
    
    return True
    
#anvil.server.wait_forever()


In [10]:
pipreqs

<module 'pipreqs' from 'C:\\Users\\JCheng10\\Anaconda3\\lib\\site-packages\\pipreqs\\__init__.py'>

In [17]:

def extract_now(path, name):
        
    pages = pdf_img(path, name)
    print("Gotten pages!")
    img = []
    for i in range(pages): 
        img.append(cv2.imread(name[:-4]+'_' + str(i+1) +'.jpg'))
    print('Converted and saved WO as image!')
    
    for i in range(len(img)):
        crop(img[i], i+1)
    img = []
    print('Cropped images!')
    
    for i in range(pages):
        pic = cv2.imread('roi' + str(i+1) +'.jpg')
        img.append(preprocess(pic))
    print('Extracted text! Putting in dataframe!')
    
    out = output_file(img, name)
    df = get_df(name[:-4]+".txt")
    print("Dataframe created!")
    
    return df


In [31]:
df = extract_now(r'C:\Users\JCheng10\OneDrive - Schlumberger\Desktop\Extract WO\Work order\WO_8628079_.pdf', 'WO_8628079_.pdf')

Gotten pages!
Converted and saved WO as image!
[[[ 105 2350 4803 2350]]

 [[ 105 2366 4804 2366]]

 [[ 105 2347 3926 2347]]]
[[[  96 2067 3798 2067]]

 [[ 141 2085 4789 2085]]]
Cropped images!
Extracted text! Putting in dataframe!
Dataframe created!


In [74]:
#df = get_df('WO_8633740_.txt')

In [79]:
#len(df)